In [1]:
include("Inheritance Util.jl")

get_decendants (generic function with 1 method)

In [2]:
# testing functions

function check_fields(typ, target_fieldnames, target_fieldtypes)
    println("Fields checked for type ", typ)
    for (tar, act) in zip(target_fieldnames, fieldnames(typ))
        if tar != act
            println(tar," != ", act)
        end
    end

    for (tar, act) in zip(target_fieldtypes, typ.types)
        if tar != act
            println(tar," != ", act)
        end
    end
end

function check_supertypes(base, supers)
    act = mi_supertypes(base)
    
    println("For type: ", base)
    
    if act != supers
        println(act, " != ", supers)
    end
end

function check_decendants(base, decendants)
     act = get_decendants(base)
    
    println("For type: ", base)
    
    if sort(act) != sort(decendants)
        println(act, " !=  ", decendants)
    end
end

check_decendants (generic function with 1 method)

In [3]:
# core inheritance

@_abstract root begin
    root_val::Int
end

@_abstract a_child root begin
    a_child_val::Char
end

@_type c_child a_child begin
    c_child_val::String
end

@_immutable i_child a_child begin
    i_child_val::Float64
end

check_fields(c_child,
             [:root_val, :a_child_val, :c_child_val],
             [Int, Char, String])

check_fields(i_child,
             [:root_val, :a_child_val, :i_child_val],
             [Int, Char, Float64])

Fields checked for type c_child
Fields checked for type i_child


In [4]:
# multiple inheritance

@_abstract root2 begin
    root2_val::Int64
end

@_type mi_child (root, root2) begin
    mi_var::Int32
end

check_fields(mi_child,
             [:root_val, :root2_val, :mi_var],
             [Int, Int64, Int32])

Fields checked for type mi_child


In [5]:
# multiple inheritance with duplicated variable

@_abstract root3 begin
    root_val::Int
end

@_type duplicated_var (root, root3) begin end

check_fields(duplicated_var, [:root_val], [Int])

Fields checked for type duplicated_var


In [6]:
# multiple inheritance with mis-matched duplicated variable
# should error

@_abstract root4 begin
    root_val::Int64
end

@_type duplicated_var_err (root, root4) begin end

check_fields(duplicated_var_err, [:root_val], [Int])

LoadError: [91msyntax: duplicate field name: "root_val" is not unique[39m

In [7]:
# create function generators for dispatch in multiple inheritance

@create_function_generator(m2,
begin
    function f1(arg1::$typ, arg2::Int)
        println("f1 Success with type ", $typ)
    end

    function f2(arg1::$typ, arg2::$typ)
        println("f2 Success with type ", $typ)
    end
end)

@m2 String

f1("test", 4)
f2("test1", "test2")

f1 Success with type String
f2 Success with type String


In [8]:
# check mi_supertypes method, both versions

check_supertypes(root, [:Any])
check_supertypes(:root, [:Any])
check_supertypes(root2, [:Any])
check_supertypes(:root2, [:Any])
check_supertypes(root3, [:Any])
check_supertypes(:root3, [:Any])
check_supertypes(a_child, [:root])
check_supertypes(:a_child, [:root])
check_supertypes(c_child, [:a_child])
check_supertypes(:c_child, [:a_child])
check_supertypes(i_child, [:a_child])
check_supertypes(:i_child, [:a_child])
check_supertypes(mi_child, [:root, :root2])
check_supertypes(:mi_child, [:root, :root2])
check_supertypes(duplicated_var, [:root, :root3])
check_supertypes(:duplicated_var, [:root, :root3])

For type: root
For type: root
For type: root2
For type: root2
For type: root3
For type: root3
For type: a_child
For type: a_child
For type: c_child
For type: c_child
For type: i_child
For type: i_child
For type: mi_child
For type: mi_child
For type: duplicated_var
For type: duplicated_var


In [9]:
# check get_subtypes

check_decendants(:root, [:a_child, :i_child, :c_child,
                         :mi_child, :duplicated_var,
                         :duplicated_var_err])
check_decendants(:a_child, [:c_child, :i_child])
check_decendants(:c_child, [])
check_decendants(:i_child, [])
check_decendants(:root2, [:mi_child])
check_decendants(:mi_child, [])
check_decendants(:root3, [:duplicated_var])
check_decendants(:duplicated_var, [])

For type: root
For type: a_child
For type: c_child
For type: i_child
For type: root2
For type: mi_child
For type: root3
For type: duplicated_var


In [10]:
# check @apply_generator

@apply_generator_to_decendants(@m2, root2)
f1(mi_child(1, 2, 3), 5)
f2(mi_child(1, 2, 3), mi_child(4, 5, 6))

f1 Success with type mi_child
f2 Success with type mi_child
